<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg" 
     width="30%" 
     align=right
     alt="Dask logo">

Custom Workloads
-------------------------

*Because not all problems are dataframes*

In [1]:
from daskernetes import KubeCluster
cluster = KubeCluster(n_workers=10)
cluster

VBox(children=(HTML(value='<b>Dashboard:</b> <a href="/user/mrocklin/proxy/8787/status" target="_blank">/user/mrocklin/proxy/8787/status</a>'), IntText(value=0, description='Requested', layout=Layout(width='150px')), Text(value='0', description='Actual', layout=Layout(width='150px')), Button(description='Scale', layout=Layout(width='150px'), style=ButtonStyle())))

In [2]:
from dask.distributed import Client, progress
c = Client(cluster)
c

Client Scheduler: tcp://10.20.57.14:39654 Dashboard: http://10.20.57.14:8787,Cluster Workers: 10 Cores: 20 Memory: 80.00 GB


In [3]:
from time import sleep

def inc(x):
    from random import random
    sleep(random())
    return x + 1

def double(x):
    from random import random
    sleep(random())
    return 2 * x
    
def add(x, y):
    from random import random
    sleep(random())
    return x + y 

In [4]:
inc(1)

2

In [5]:
future = c.submit(inc, 1)  # returns immediately with pending future
future

<Future: status: pending, key: inc-06206684d462f6701bb4fc3baa4258ec>

In [6]:
future  # scheduler and client talk constantly

<Future: status: finished, type: int, key: inc-06206684d462f6701bb4fc3baa4258ec>

In [7]:
future.result()

2

### Submit many tasks

We submit many tasks that depend on each other in a normal Python for loop

In [8]:
%%time
zs = []
for i in range(256):
    x = c.submit(inc, i)     # x = inc(i)
    y = c.submit(double, x)  # y = inc(x)
    z = c.submit(add, x, y)  # z = inc(y)
    zs.append(z)
    
total = c.submit(sum, zs)

CPU times: user 327 ms, sys: 29 ms, total: 356 ms
Wall time: 328 ms


In [ ]:
total.result()

### Custom computation: Tree summation

As an example of a non-trivial algorithm, consider the classic tree reduction.  We accomplish this with a nested for loop and a bit of normal Python logic.

```
finish           total             single output
    ^          /        \
    |        c1          c2        neighbors merge
    |       /  \        /  \
    |     b1    b2    b3    b4     neighbors merge
    ^    / \   / \   / \   / \
start   a1 a2 a3 a4 a5 a6 a7 a8    many inputs
```

In [ ]:
L = zs
while len(L) > 1:
    new_L = []
    for i in range(0, len(L), 2):
        future = c.submit(add, L[i], L[i + 1])  # add neighbors
        new_L.append(future)
    L = new_L                                   # swap old list for new
   

In [ ]:
del L

### Visualize Computation

In [ ]:
from dask import delayed, visualize

L = zs
while len(L) > 1:
    new_L = []
    for i in range(0, len(L), 2):
        future = delayed(add)(L[i], L[i + 1])  # add neighbors
        new_L.append(future)
    L = new_L                                   # swap old list for new
    
visualize(*L)